In [ ]:
# Setting up the Colab environment. DO NOT EDIT!
import os
import warnings
warnings.filterwarnings("ignore")

try:
    import otter

except ImportError:
    ! pip install -q otter-grader==4.0.0
    import otter

if not os.path.exists('walkthrough-tests'):
    zip_files = [f for f in os.listdir() if f.endswith('.zip')]
    assert len(zip_files)>0, 'Could not find any zip files!'
    assert len(zip_files)==1, 'Found multiple zip files!'
    ! unzip {zip_files[0]}

grader = otter.Notebook(colab=True,
                        tests_dir = 'walkthrough-tests')

# Walkthrough

## Introduction

Get ready to dive into some data analysis as we explore the effectiveness of a hypothetical HIV treatment trial.
In this walkthrough, we have a dataset containing information from 30 people living with HIV (PLWH) who were randomly assigned to a treatment or control group.
After receiving the treatment, they stopped their ART and were monitored weekly for the number of weeks until their first "detectable" viral load was found.
We will use `Pandas` to analyze this data and evaluate the treatment's effectiveness.
By the end of this activity, you will be proficient in loading spreadsheet data into Python, creating derived columns in `DataFrames`, and using summary methods like sum, mean, and max.
Let's get started!

## Learning Objectives
At the end of this learning activity you will be able to:
 - Practice loading spreadsheet data into Python using `pandas`.
 - Use Python methods to create derived columns in `pd.DataFrames`.
 - Use `Pandas` summary methods like sum, mean, and max.
 - Employ basic filtering and data extraction from `pandas`.

## Dataset Reference

_File_: `trial_data.csv`

_Columns_:

 - `age` : (years) Current age during the study. 
 - `age_initial_infection` : (years) Age at which the participant was initially infected.
 - `initial_viral_load` : (copies/ul) The level of infection at the start of the study.
 - `treatment` : (boolean) `True` for participant in the treatment group, `False` for those in the control group.
 - `weeks_to_failure` : (weeks) Time from the treatment to the first week of uncontrolled viral load.



## Imports

While _basic_ Python can do a lot, you have to do everything yourself.
The **real** power of Python is that you can `import` code that is written by others.

For this course, we will use a common data science stack of interoperable tools centered around the [Numpy](https://numpy.org/).

There are four that we will use regularly, two of which we'll cover today.

### Numpy

[Numpy](https://numpy.org/)

A numerical Python library that contains incredibly fast arrays, mathematical functions, and other useful utilities.

By convention, the community tends to _alias_ the long `numpy` as `np`.

In [ ]:
import numpy as np

### Pandas

[Pandas](https://pandas.pydata.org/)

A libary that sits atop `numpy` and provides a _spreadsheet_ style object called a `DataFrame` along with a plethora of data sciecne utilities.
This is the main tool we will be using for data exploration.

By convention, the community tends to _alias_ the long `pandas` as `pd`.

In [ ]:
import pandas as pd

Nicely, it can read `csv` files for us.

In [ ]:
trial_df = pd.read_csv('trial_data.csv')

# If a `DataFrame` is the last line, it will display a nice summary
trial_df

```{note}
:class: dropdown
I often use the suffix `_df` when I create `pd.DataFrames`.
It is not required, but utilizing naming conventions makes your code easier to understand by yourself and others.
```

And we should see that this exactly matches the table we saw in Excel.

The object we got back is called a `DataFrame`.

In [ ]:
type(trial_df)

If we only want to see a small version of the `DataFrame` we can use the `.head()` _method_.

In [ ]:
trial_df.head()

## Acting on Columns

We can reference each column by name using square brackets `[]`.
For example: Extracting the `age` column like so:

In [ ]:
trial_df['age']

### Q1: Extract the `initial_viral_load` column ?

|               |    |
| --------------|----|
| Points        | 2  |
| Public Checks | 2  |
| Hidden Tests  | 0  |

_Points:_ 2

In [ ]:
init_vl = ...

In [ ]:
grader.check("q1_init_vl")

Once we can extract columns, we can start summarizing them.

In [ ]:
age_col = trial_df['age']
age_mean = age_col.mean()
print(f'The mean age of the population is {age_mean:0.1f} yrs.')

Expressions can also be _chained_. 
They are functionally the same, the only difference is aesthetic. 

In [ ]:
age_mean_short = trial_df['age'].mean()
print(f'The mean age of the population is {age_mean_short:0.1f} yrs, even when done on a single line.')

### Q2: Calculate the average `weeks_to_failure` for the whole population?



|               |    |
| --------------|----|
| Points        | 2  |
| Public Checks | 2  |
| Hidden Tests  | 0  |

_Points:_ 2

In [ ]:
average_weeks = ...

In [ ]:
grader.check("q2_pop_weeks_to_failure")

We can also summarize an entire `DataFrame` with a single command.

In [ ]:
trial_df.mean()

In this case the summary went _down_ the columns and calculated a mean for each.

There are a number of other summarization _methods_.
 - `max()`
 - `min()`
 - `mode()`
 - `median()`
 - `var()`
 - `std()`
 - `nunique()`

```{note}
:class: dropdown
Methods, are functions that are attached to an `object`.
They usually act on the object to provide a summary, perform a transformation, or otherwise utilize the information within the object.
In this case, these summarization methods utilize the information within the `trial_df` dataframe to summarize each column.
```

In [ ]:
trial_df.describe()

Selecting columns is nice.
We can also add a new column based on another one.

In HIV research it is often important to know how long someone has been living with HIV.
However, this dataset contains their current age, and their age at infection.
We can use these two to calculate the length.

In [ ]:
# first make a new `Series`
years_infected = trial_df['age'] - trial_df['age_initial_infection']

# Then add that series into the table
trial_df['years_infected'] = years_infected
trial_df.head()

In [ ]:
# Alternatively
trial_df['years_infected'] = trial_df['age'] - trial_df['age_initial_infection']
trial_df.head()

## Acting on Rows

### Indexing

When selecting rows, or rows and columns, we need to use the `.loc` attribute of the `DataFrame`.

We can select by row number.

In [ ]:
trial_df.loc[0]

In [ ]:
# We can use a : to indicate a range.
trial_df.loc[0:10]

In [ ]:
# We can provide an arbitrary list
trial_df.loc[[0, 5, 7, 13]]

In [ ]:
# We can also select columns at the same time.
trial_df.loc[[0, 5, 7, 13], ['initial_viral_load', 'age']]

### Boolean Indexing

If we do not know the row number ahead of time, but instead want to select rows based on their values, we can using boolean indexing.
In this stragey we create a new `pd.Series` of True/False values where True corresponds to the ones we want.

Start by finding everyone over 50 years old.

In [ ]:
age_mask = trial_df['age'] > 50
aged_samples = trial_df.loc[age_mask]
aged_samples.head()

```{note}
:class: dropdown
I often use the suffix `_mask` when I create boolean indexes.
It is not required, but utilizing naming conventions makes your code easier to understand by yourself and others.
```

Now, if we also wanted to split by the initial_viral_load we might do:

In [ ]:
high_vl_mask = trial_df['initial_viral_load'] > 50

In [ ]:
aged_high_vl = trial_df.loc[age_mask & high_vl_mask]
aged_high_vl.head()

In [ ]:
# ~ can be used to say "not"
aged_low_vl = trial_df.loc[age_mask & ~high_vl_mask]
aged_low_vl.head()

### Q3: Calculate the average weeks to failure for the treated population?

|               |    |
| --------------|----|
| Points        | 2  |
| Public Checks | 2  |
| Hidden Tests  | 0  |

_Points:_ 2

In [ ]:
treated_average_weeks = ...

In [ ]:
grader.check("q3_treated_weeks_to_failure_indexing")

Utilizing boolean indexing you can express _any_ algorithmic row selecting strategy.
This can even include comparisons between rows, for example if there were multiple rows of the same sample.
We will cover these strategies later in the course.

Sometimes, our searches are simple.
Pandas also includes another method for indexing rows called `.query()` for these purposes.

### Querying

`.query()` is an interface that facilitates simple queries qith a few specific limitations:
 - It can only use the information present in the row.
 - It can only work on one row at a time.
 - Column headers cannot contain spaces, dots, dashes, commas, or emoji.

Our questions on this dataset easily fit within those constraints.

In [ ]:
# All treatment rows
trial_df.query('treatment == True').head()

In [ ]:
trial_df.query('treatment == False').head()

You can also make them more complex.

In [ ]:
trial_df.query('age > 33 & treatment == True')

This statement doesn't make a "biological sense", but it is an example of a valid comparison.

In [ ]:
trial_df.query('age >= initial_viral_load')

### Q4: Calculate the average `weeks_to_failure` for the untreated population?



|               |    |
| --------------|----|
| Points        | 2  |
| Public Checks | 2  |
| Hidden Tests  | 0  |

_Points:_ 2

In [ ]:

untreated_average_weeks = ...

In [ ]:
print(f'Untreated participants took {untreated_average_weeks:0.1f} weeks to rebound.')

In [ ]:
grader.check("q4_untreated_weeks_to_failure")

### Q4: Calculate the average `weeks_to_failure` for the treated population?



|               |    |
| --------------|----|
| Points        | 2  |
| Public Checks | 2  |
| Hidden Tests  | 0  |

_Points:_ 2

In [ ]:

treated_average_weeks = ...

In [ ]:
print(f'Treated patients took {treated_average_weeks:0.1f} weeks to rebound.')

In [ ]:
grader.check("q4_treated_weeks_to_failure")

# Conclusion

We can see that this treatment extended the average time off ART from ~3 weeks to ~7 weeks.
While not a complete cure, any incremental step is useful progress in the elimination of HIV.

In the lab you will use similar techniques to explore whether other factors in this dataset impact the results.
In future weeks we will explore statistical techniques to understand whether this difference is due to chance, or due to the effect of the treatment.

## Submission

You do not need to submit this walkthrough notebook.
Simply complete the quiz.

---------------------------------------------